In [1]:
import os
import gym
import cv2
import argparse
import sys, glob
import numpy as np
import pandas as pd
import pdb
import keras
from keras import backend as K
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Reshape
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam, Adamax, RMSprop
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation, Dropout, Flatten
from keras.layers.convolutional import UpSampling2D, Convolution2D
IMAGE_LENGTH = int(33600)
epochs = 10


Using TensorFlow backend.


In [0]:
def load_data(path):
    #pdb.set_trace()
    df = pd.read_csv(path, sep = '.', header = None)
    letters = df[(df.index % (IMAGE_LENGTH+1)==0)].values.tolist()
    images = df[(df.index % (IMAGE_LENGTH+1)!=0)].values.tolist()
    n =IMAGE_LENGTH
    final = [images[i * n:(i + 1) * n] for i in range((len(images) + n - 1) // n )]  
    #pdb.set_trace()
    return letters, final

In [0]:
# Prepare first dataset 
letter, final = load_data('/content/drive/My Drive/DATOS/EIT/NICE ACADEMIC/AI/project/dataset_thomas_v1.txt')
for i in range (0, len(final)):
    final[i] = np.concatenate(final[i])
#pdb.set_trace()

letter = np.concatenate(letter)
final = np.array(final)
letter=letter.reshape(1, len(letter))
letter = letter[0, :]

In [4]:
# Prepare  second dataset 
letter2, final2 = load_data('/content/drive/My Drive/DATOS/EIT/NICE ACADEMIC/AI/project/dataset_v2.txt')
for i in range (0, len(final2)):
    final2[i] = np.concatenate(final2[i])
#pdb.set_trace()

letter2 = np.concatenate(letter2)
final2 = np.array(final2)
letter2=letter2.reshape(1, len(letter2))
letter2 = letter2[0, :]

FileNotFoundError: ignored

In [0]:
# Prepare third dataset
# Prepare  second dataset 
letter3, final3 = load_data('/content/drive/My Drive/DATOS/EIT/NICE ACADEMIC/AI/project/dataset_thomas_v3.txt')
for i in range (0, len(final3)):
    final3[i] = np.concatenate(final3[i])
#pdb.set_trace()

letter3 = np.concatenate(letter3)
final3 = np.array(final3)
letter3=letter3.reshape(1, len(letter3))
letter3 = letter3[0, :]

In [0]:
# Prepare third dataset
# Prepare  second dataset 
letter4, final4 = load_data('/content/drive/My Drive/DATOS/EIT/NICE ACADEMIC/AI/project/dataset_haider_v1.txt')
for i in range (0, len(final4)):
    final4[i] = np.concatenate(final4[i])
#pdb.set_trace()

letter4 = np.concatenate(letter4)
final4 = np.array(final4)
letter4=letter4.reshape(1, len(letter4))
letter4 = letter4[0, :]

In [0]:
final = np.concatenate((final, final2, final3, final4))

In [0]:
letter = np.concatenate((letter, letter2, letter3, letter4))

In [0]:
final.shape

In [0]:
# Split dataset using a rule of 0.7
train_ratio = 0.7

n_train_samples = int(len(final) * train_ratio)
x_train, y_train = final[:n_train_samples], letter[:n_train_samples]
x_val, y_val = final[n_train_samples:], letter[n_train_samples:]


In [0]:
num_classes = 9
#x_train /= 255
#x_val /= 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)


In [0]:
# Input shape will be determined by x_train
model = Sequential()
model.add(Dense(150, activation='sigmoid', input_shape=(33600,)))
model.add(Dense(200, activation='sigmoid'))
model.add(Dense(240, activation='sigmoid'))
model.add(Dense(400, activation='sigmoid'))
model.add(Dense(333, activation='sigmoid'))
model.add(Dense(231, activation='sigmoid'))
model.add(Dense(147, activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(40, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='nadam',
              metrics=['accuracy'])
#pdb.set_trace()
history = model.fit(x_train, y_train,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val))
score = model.evaluate(x_val, y_val, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])
# This code saves the weights
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_weights.h5")
print("Saved model to disk")
